In [39]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN
from sklearn.metrics import adjusted_rand_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [40]:
df = pd.read_pickle('crowd_train_all_data_embedded.pkl')


def get_dataframe(df, min_count, max_count):
    df_temp = df.copy()

    freq = df_temp.source_id.value_counts()
    freq = freq[min_count <= freq]
    df_temp = df_temp[df.source_id.isin(freq.index)]

    df_temp = (
        df_temp.groupby('source_id')
        .apply(lambda x: x.nlargest(max_count, 'duration'))
        .reset_index(drop=True)
        .drop(
            columns=[
                'duration',
                'hash_id',
                'annotator_emo',
                'golden_emo',
                'annotator_id',
                'speaker_text',
                'speaker_emo',
            ]
        )
    )

    df_temp.dropna(inplace=True)

    print(df_temp.source_id.nunique())

    return df_temp


df_t = get_dataframe(df, 350, 350)

8


In [50]:
X = np.vstack(df_t.audio_feature.to_numpy()).reshape(df_t.shape[0], -1)
le = LabelEncoder()
y = le.fit_transform(df_t.source_id)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train.shape, X_test.shape, len(y_train[y_train == 4]), len(y_test[y_test == 4])

((2240, 498), (560, 498), 280, 70)

In [51]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [52]:
X_train.shape, y_train.shape

((2240, 498), (2240,))

In [53]:
df_train = pd.DataFrame(np.concatenate([X_train_scaled, y_train.reshape(-1, 1)], axis=1))
clusters_centres = df_train.drop_duplicates(subset=[498]).drop(columns=[498]).to_numpy()

# Модели с заранее известным количеством кластеров

## kmeans

In [91]:
params = {
    "n_clusters": len(set(y)),
    "random_state": 42,
    # "max_iter": 10_000,
    # "algorithm": "elkan",
    "init": 'k-means++',
    # "tol": 0.01,
}
sk_kmeans = KMeans(**params)
sk_kmeans.fit(X_train_scaled)

C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


KMeans(n_clusters=102, random_state=42)

In [92]:
preds_train = sk_kmeans.predict(X_train_scaled)
preds_test = sk_kmeans.predict(X_test_scaled)
kmeans_f1_w_train = f1_score(y_train, preds_train, average='weighted')
kmeans_f1_w_test = f1_score(y_test, preds_test, average='weighted')
kmeans_f1_w_train, kmeans_f1_w_test

(0.001777319114780415, 0.0016339869281045754)

In [94]:
import mlflow
from sklearn.preprocessing import StandardScaler

from sklearn import metrics

mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment = mlflow.set_experiment(f"clusterization, 102 classes, 5 per class")

run_name = "kmeans normalized centres k-means++"

with mlflow.start_run(run_name=run_name) as run:
    params = {
    "n_clusters": len(set(y)),
    "random_state": 42,
    # "max_iter": 10_000,
    # "algorithm": "elkan",
    "init": 'k-means++',
    # "tol": 0.01,
    }
    sk_kmeans = KMeans(**params)
    sk_kmeans.fit(X_train_scaled)

    predicts = sk_kmeans.predict(X_test_scaled)

    mlflow.log_metric("train f1_weighted", metrics.f1_score(y_train, sk_kmeans.predict(X_train_scaled), average="weighted"))
    mlflow.log_metric("f1_weighted", metrics.f1_score(y_test, predicts, average="weighted"))
    mlflow.log_metric("f1_micro", metrics.f1_score(y_test, predicts, average="micro"))
    mlflow.log_metric("accuracy", metrics.accuracy_score(y_test, predicts))
    
    mlflow.log_params(params)

C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


In [70]:
params = {
    "n_clusters": len(set(y)),
    "random_state": 42,
    # "max_iter": 10_000,
    # "algorithm": "elkan",
    "init": clusters_centres,
    # "tol": 0.01,
}
sk_kmeans = KMeans(**params)
sk_kmeans.fit(X_train_scaled)

C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1416: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(init=array([[-0.69098183, -0.46656332, -0.38895905, ...,  1.35435952,
         0.5239523 , -0.61049377],
       [ 0.49568172,  2.39631479,  2.25760068, ..., -0.01618921,
        -1.38158758,  0.1227285 ],
       [ 0.41884317,  0.44148583,  0.00325378, ..., -1.04030161,
        -0.95094262, -1.39637738],
       ...,
       [-0.97756849, -1.339689  , -1.18725004, ...,  0.2163377 ,
        -1.70038313, -0.77205257],
       [ 0.17963005, -0.01272553, -0.34096361, ..., -1.60433391,
        -0.78617176, -1.48488896],
       [-0.11380994, -1.56222909, -2.21851011, ...,  0.8176524 ,
         0.024701  , -0.01381242]]),
       random_state=42)

In [71]:
preds_train = sk_kmeans.predict(X_train_scaled)
preds_test = sk_kmeans.predict(X_test_scaled)
kmeans_f1_w_train = f1_score(y_train, preds_train, average='weighted')
kmeans_f1_w_test = f1_score(y_test, preds_test, average='weighted')
kmeans_f1_w_train, kmeans_f1_w_test

(0.19033971923419596, 0.19371523308166683)

## AgglomerativeClustering

In [67]:
sk_ac = AgglomerativeClustering(n_clusters=len(set(y)), linkage='ward')

sk_ac_preds = sk_ac.fit_predict(X)

sk_ac_f1_w = f1_score(y, sk_ac_preds, average='weighted')
sk_ac_f1_w

0.1823282267597413

In [95]:
import mlflow
from sklearn.preprocessing import StandardScaler

from sklearn import metrics

mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment = mlflow.set_experiment(f"clusterization, 102 classes, 5 per class")

run_name = "Agglomerative Clustering ward"

with mlflow.start_run(run_name=run_name) as run:
    params = {
    "n_clusters": len(set(y)),
    "linkage": 'ward'
    }
    sk_ac = AgglomerativeClustering(**params)

    predicts = sk_ac.fit_predict(X)

    mlflow.log_metric("f1_weighted", metrics.f1_score(y, predicts, average="weighted"))
    mlflow.log_metric("f1_micro", metrics.f1_score(y, predicts, average="micro"))
    mlflow.log_metric("accuracy", metrics.accuracy_score(y, predicts))
    
    mlflow.log_params(params)

## SpectralClustering

In [81]:
sk_ac = SpectralClustering(n_clusters=len(set(y)), gamma=10, random_state=0)

sk_ac_preds = sk_ac.fit_predict(X)

sk_ac_f1_w = f1_score(y, sk_ac_preds, average='weighted')
sk_ac_f1_w

C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\sklearn\manifold\_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


0.11766561309783533

In [96]:
import mlflow
from sklearn.preprocessing import StandardScaler

from sklearn import metrics

mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment = mlflow.set_experiment(f"clusterization, 102 classes, 5 per class")

run_name = "Spectral Clustering"

with mlflow.start_run(run_name=run_name) as run:
    params = {
    "n_clusters": len(set(y)),
    'random_state': 0,
    }
    sk_ac = SpectralClustering(**params)

    predicts = sk_ac.fit_predict(X)

    mlflow.log_metric("f1_weighted", metrics.f1_score(y, predicts, average="weighted"))
    mlflow.log_metric("f1_micro", metrics.f1_score(y, predicts, average="micro"))
    mlflow.log_metric("accuracy", metrics.accuracy_score(y, predicts))
    
    mlflow.log_params(params)

C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\sklearn\manifold\_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


# Модели без заранее заданного количества классов

## ESoinn

In [45]:
from essoin_model.esoinn import ESoinn

s = ESoinn(iteration_threshold=300, plt_in_fit=False)
s.fit(X)

 10%|▉         | 276/2800 [00:00<00:04, 518.61it/s] 

[False, True, True, True, False, True, False, True, True, True, False, False, False, True, False, False, True, False, True, False, False, False, True, True, False, False, False, False, False, False, False, True, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, True, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False,

 15%|█▍        | 410/2800 [00:01<00:06, 344.72it/s]

Number of classes： 10


 20%|██        | 560/2800 [00:01<00:06, 359.09it/s]

...

[False, False, False, False, False, True, False, False, False, True, False, True, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

100%|██████████| 2800/2800 [00:14<00:00, 197.14it/s]

Number of classes： 12
Number of classes： 103


<Figure size 432x288 with 0 Axes>

ESoinn(dim=498, iteration_threshold=300, plt_in_fit=False)

In [46]:
essoin_preds = s.predict(X)
essoin_score = adjusted_rand_score(y, essoin_preds)
essoin_score

Number of classes： 103


  0%|          | 0/2800 [00:00<?, ?it/s]

100%|██████████| 2800/2800 [00:00<00:00, 2823.33it/s]


0.2775579507089917

In [48]:
import mlflow
from sklearn.preprocessing import StandardScaler
from essoin_model.esoinn import ESoinn

from sklearn import metrics

mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment = mlflow.set_experiment(f"clusterization without the number of classes, 8 classes")

run_name = "ESoinn"

with mlflow.start_run(run_name=run_name) as run:
    params = {
        "iteration_threshold": 300,
        "plt_in_fit": False,
    }
    s = ESoinn(**params)
    s.fit(X)

    essoin_preds = s.predict(X)
    mlflow.log_metric("adjusted_rand_score", adjusted_rand_score(y, essoin_preds))

    mlflow.log_params(params)


  9%|▉         | 262/2800 [00:00<00:04, 579.39it/s]

[True, True, False, True, True, True, False, False, False, False, True, True, False, True, False, False, False, False, False, False, True, True, False, True, True, False, False, True, False, False, True, False, False, False, False, False, True, False, False, True, True, False, True, False, False, False, False, False, False, False, True, True, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Fal

 13%|█▎        | 377/2800 [00:01<00:07, 313.42it/s]

Number of classes： 5


 20%|██        | 574/2800 [00:01<00:06, 350.91it/s]

...

100%|██████████| 2800/2800 [00:19<00:00, 141.69it/s]

Number of classes： 89


<Figure size 432x288 with 0 Axes>

Number of classes： 89


100%|██████████| 2800/2800 [00:00<00:00, 3160.76it/s]


## DBScan

In [54]:
sk_dbscan = DBSCAN(eps=0.3, min_samples=3)
sk_dbscan_pred_res = sk_dbscan.fit_predict(X)
sk_dbscan_ari = adjusted_rand_score(y, sk_dbscan_pred_res)
print(f'Adjusted Rand Score for sk DBSCAN: {sk_dbscan_ari}', '', sep='\n')
print('prediction', sk_dbscan_pred_res, sep='\n')

Adjusted Rand Score for sk DBSCAN: 0.0

prediction
[-1 -1 -1 ... -1 -1 -1]


In [55]:
import mlflow
from sklearn.preprocessing import StandardScaler
from essoin_model.esoinn import ESoinn

from sklearn import metrics

mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment = mlflow.set_experiment(f"clusterization without the number of classes, 8 classes")

run_name = "DBSCAN"

with mlflow.start_run(run_name=run_name) as run:
    params = {
        "eps": 0.3, 
        "min_samples": 3,
    }
    s = DBSCAN(**params)
    preds = s.fit_predict(X)
    mlflow.log_metric("adjusted_rand_score", adjusted_rand_score(y, preds))
    
    mlflow.log_params(params)